In [1]:


import matplotlib.pyplot as plt

from chord import Chord
# load 
import sys
import time

from sklearn.model_selection import cross_val_score,train_test_split
from datetime import date
import numpy as np
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.io import loadmat
from scipy.stats import mannwhitneyu
from statsmodels.stats.multitest import multipletests
import pingouin as pg
import subprocess
import os 
from scipy.stats import ranksums
from scipy.stats import pearsonr
from scipy import stats


if sys.platform == 'darwin':
    print("Current system is macOS")
    main_fold_path = '/Users/shanxiafeng/Documents/Project/Research/fnirs-prognosis/code/fnirs-treatment-response-prediction'
elif sys.platform == 'linux':
    print("Current system is Ubuntu")
    main_fold_path = '/home/jy/Documents/fnirs/treatment_response/fnirs-depression-deeplearning'
else:
    print("Current system is neither macOS nor Ubuntu")
    
    
os.chdir(main_fold_path)

from utils.fnirs_utils import get_channel_to_region_dict
    
DATA =  np.load('allData/prognosis_mix_hb/pretreatment_response/hb_data.npy')
LABEL =  np.load('allData/prognosis_mix_hb/pretreatment_response/label.npy')
data = DATA
label = LABEL

x = np.load('allData/prognosis_mix_hb/pretreatment_response/label_hamd.npy')
reduced_percentage = (x[:, 1] - x[:, 0]) / x[:, 0]

HbO = np.transpose(data[...,0::2],(0,2,1))
# HbO = individual_normalization(HbO)
print(f'HbO: {HbO.shape}')
HbR = np.transpose(data[...,1::2],(0,2,1))
HbT = HbO + HbR


label_hamd = np.load('allData/prognosis_mix_hb/pretreatment_response/label_hamd.npy')
reduced_percentage = (label_hamd[:, 1] - label_hamd[:, 0]) / label_hamd[:, 0]

Current system is Ubuntu
HbO: (64, 1250, 52)


| Channel | Region    | Task change(R) | Mean(R) | Task activation(R) | Task change(P) | Mean(P) | Task activation(P) |
|-------------|-----------|-----------------|------------|------------------|


In [5]:
channel_to_dict =  get_channel_to_region_dict()


HB = HbO  
y_label = reduced_percentage # reduced_percentage or label
hb_dict = {
    'HbO': HbO,
    'HbR': HbR,
    'HbT': HbT
}
# feature_extraction - task activation
def extracting_task_activation_feature(HB, task_start_index, task_end_index, feature_name, i):
    if feature_name == 'task_change': x = HB[:, task_end_index:, i].mean(axis=1) - HB[:, :task_start_index, i].mean(axis=1)
    if feature_name == 'mean': x = HB[:, :, i].mean(axis=1) 
    if feature_name == 'task_activation': x = HB[:, task_start_index:task_end_index, i].mean(axis=1) - HB[:, :task_start_index, i].mean(axis=1) - HB[:, task_end_index:, i].mean(axis=1)
    return x 
        
def get_r_and_p_of_hb_feature_using_pearsonr(HB, y_label, feature_name_arr, ch):
    result = []
    for feature_name in feature_name_arr:
        x = extracting_task_activation_feature(HB, 100, 700, feature_name, ch)
        r, p = pearsonr(x, y_label)
        result.append(r)
        result.append(p)
    return result
for hb_type, HB in hb_dict.items():
    
    print(f'Correlation of {hb_type} features and HAMD reduction percentage. * means a statistical difference (P<0.05), corrected with FDR at α = 0.05')
    print()
    print("| Channel | Region    | Task change(R) | Mean(R) | Task activation(R) | Task change(P) | Mean(P) | Task activation(P) |")
    print("|-------------|-----------|-----------------|------------|------------------|-----------------|------------|------------------|")
    
    all_p = {'p1':[], 'p2':[], 'p3':[]}
    for ch in range(52):
        region_name = channel_to_dict[ch+1]
        region_name = region_name.replace('_',' ')
        r1, p1, r2, p2, r3, p3 = get_r_and_p_of_hb_feature_using_pearsonr(HB, y_label, ['task_change', 'mean', 'task_activation'], ch)
        all_p['p1'].append(p1)
        all_p['p2'].append(p2)
        all_p['p3'].append(p3)
        # print(f"| {ch+1} | {region_name} | {r1:.4f} | {r2:.4f} | {r3:.4f} | {p1:.4f}{'*' if p1 < 0.05 else ''} | {p2:.4f}{'*' if p2 < 0.05 else ''} | {p3:.4f}{'*' if p3 < 0.05 else ''} |")
    
    for key, item in all_p.items():
        reject, p_corrected, _, _ = multipletests(item, alpha=0.05, method='fdr_bh')
        all_p[key] = p_corrected
    
    for ch in range(52):
        region_name = channel_to_dict[ch+1]
        region_name = region_name.replace('_',' ')
        r1, p1, r2, p2, r3, p3 = get_r_and_p_of_hb_feature_using_pearsonr(HB, y_label, ['task_change', 'mean', 'task_activation'], ch)
        p1 = all_p['p1'][ch]
        p2 = all_p['p2'][ch]
        p3 = all_p['p3'][ch]
        print(f"| {ch+1} | {region_name} | {r1:.4f} | {r2:.4f} | {r3:.4f} | {p1:.4f}{'*' if p1 < 0.05 else ''} | {p2:.4f}{'*' if p2 < 0.05 else ''} | {p3:.4f}{'*' if p3 < 0.05 else ''} |")


Correlation of HbO features and HAMD reduction percentage. * means a statistical difference (P<0.05), corrected with FDR at α = 0.05

| Channel | Region    | Task change(R) | Mean(R) | Task activation(R) | Task change(P) | Mean(P) | Task activation(P) |
|-------------|-----------|-----------------|------------|------------------|-----------------|------------|------------------|
| 1 | right PSFC | 0.0889 | -0.0681 | 0.0281 | 0.7413 | 0.9032 | 0.9956 |
| 2 | right PSFC | -0.2695 | 0.3245 | -0.3027 | 0.2905 | 0.4629 | 0.5558 |
| 3 | right DPC | 0.2022 | -0.0469 | 0.2054 | 0.3658 | 0.9032 | 0.5977 |
| 4 | right DPC | 0.2252 | -0.1075 | 0.2278 | 0.2942 | 0.8998 | 0.5558 |
| 5 | MPC | 0.2312 | 0.0946 | 0.0007 | 0.2905 | 0.9032 | 0.9956 |
| 6 | MPC | -0.0655 | 0.0641 | -0.0653 | 0.7413 | 0.9032 | 0.9956 |
| 7 | left DPC | 0.2304 | 0.1960 | -0.1481 | 0.2905 | 0.8297 | 0.9956 |
| 8 | left DPC | -0.2382 | 0.1936 | -0.2072 | 0.2905 | 0.8297 | 0.5977 |
| 9 | left PSFC | 0.0692 | 0.0097 | 0.0251 |

: 